<center>
<img src="../../img/ods_stickers.jpg" />
    
## [mlcourse.ai](https://mlcourse.ai) – Open Machine Learning Course 
Author: [Yury Kashnitskiy](https://yorko.github.io) (@yorko). Edited by Sergey Kolchenko (@KolchenkoSergey). This material is subject to the terms and conditions of the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license. Free use is permitted for any non-commercial purpose.

## <center>Assignment #6
### <center> Beating baselines in "How good is your Medium article?"
    
<img src='../../img/medium_claps.jpg' width=40% />


[Competition](https://www.kaggle.com/c/how-good-is-your-medium-article). The task is to beat "A6 baseline" (~1.45 Public LB score). Do not forget about our shared ["primitive" baseline](https://www.kaggle.com/kashnitsky/ridge-countvectorizer-baseline) - you'll find something valuable there.

**Your task:**
 1. "Freeride". Come up with good features to beat the baseline "A6 baseline" (for now, public LB is only considered)
 2. You need to name your [team](https://www.kaggle.com/c/how-good-is-your-medium-article/team) (out of 1 person) in full accordance with the [course rating](https://drive.google.com/open?id=19AGEhUQUol6_kNLKSzBsjcGUU3qWy3BNUg8x8IFkO3Q). You can think of it as a part of the assignment. 16 credits for beating the mentioned baseline and correct team naming.
 
*For discussions, please stick to [ODS Slack](https://opendatascience.slack.com/), channel #mlcourse_ai, pinned thread __#a6__*

In [3]:
import os
import json
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

The following code will help to throw away all HTML tags from an article content.

In [4]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

Supplementary function to read a JSON line without crashing on escape characters.

In [5]:
def read_json_line(line=None):
    result = None
    try:        
        result = json.loads(line)
    except Exception as e:      
        # Find the offending character index:
        idx_to_replace = int(str(e).split(' ')[-1].replace(')',''))      
        # Remove the offending character:
        new_line = list(line)
        new_line[idx_to_replace] = ' '
        new_line = ''.join(new_line)     
        return read_json_line(line=new_line)
    return result

Extract features `content`, `published`, `title` and `author`, write them to separate files for train and test sets.

In [12]:
# %cd ..

/Users/admin/PycharmProjects/ods/mlcourse.ai/jupyter_english/assignments_fall2018


In [11]:
# !python setup.py install

running install
running bdist_egg
running egg_info
creating googletrans.egg-info
writing googletrans.egg-info/PKG-INFO
writing dependency_links to googletrans.egg-info/dependency_links.txt
writing requirements to googletrans.egg-info/requires.txt
writing top-level names to googletrans.egg-info/top_level.txt
writing manifest file 'googletrans.egg-info/SOURCES.txt'
reading manifest file 'googletrans.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'googletrans.egg-info/SOURCES.txt'
installing library code to build/bdist.macosx-10.7-x86_64/egg
running install_lib
running build_py
creating build/bdist.macosx-10.7-x86_64/egg
creating build/bdist.macosx-10.7-x86_64/egg/googletrans
copying build/lib/googletrans/compat.py -> build/bdist.macosx-10.7-x86_64/egg/googletrans
copying build/lib/googletrans/models.py -> build/bdist.macosx-10.7-x86_64/egg/googletrans
copying build/lib/googletrans/client.py -> build/bdist.macosx-10.7-x86_64/egg/googletrans
copying bui

In [9]:
from googletrans import Translator

In [10]:
translator = Translator()
translator.translate('veritas lux mea').text

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [6]:
def extract_features_and_write(path_to_data,
                               inp_filename, translator, is_train=True):
    
    features = ['title']
    prefix = 'train' if is_train else 'test'
    feature_files = [open(os.path.join(path_to_data,
                                       '{}_{}.txt'.format(prefix, feat)),
                          'w', encoding='utf-8')
                     for feat in features]
    
    with open(os.path.join(path_to_data, inp_filename), 
              encoding='utf-8') as inp_json_file:

        for line in tqdm_notebook(inp_json_file):
            json_data = read_json_line(line)
            for i in range(1):
                data = json_data[features[i]]
                if (type(data) is dict):
                    for key, value in data.items():
                        if not (value is None):
                            clean_value = strip_tags(value).replace('\n', ' ').replace('\r', ' ')
                            feature_files[i].write(key + ":" + clean_value + '||')
#                             try:
#                                 feature_files[i].write(key + ":" + translator.translate(clean_value).text + '||')
#                             except:
#                                 feature_files[i].write(key + ":" + clean_value + '||')
                        else:
                            feature_files[i].write(key + ":" + '-' + '||')
                    feature_files[i].write('\n')
                else:
                    clean_data = strip_tags(data).replace('\n', ' ').replace('\r', ' ')
                    feature_files[i].write(clean_data + '\n')
#                     try:
#                         eng_data = translator.translate(clean_data).text
#                         feature_files[i].write(eng_data + '\n')
#                         print(eng_data)
#                     except:
#                         feature_files[i].write(clean_data + '\n')
#                         print(clean_data)
                    
            # You code here

In [7]:
PATH_TO_DATA = '../../data' # modify this if you need to

In [11]:
extract_features_and_write(PATH_TO_DATA, 'train.json', translator, is_train=True)

In [8]:
extract_features_and_write(PATH_TO_DATA, 'test.json', is_train=False)

**Add the following groups of features:**
    - Tf-Idf with article content (ngram_range=(1, 2), max_features=100000 but you can try adding more)
    - Tf-Idf with article titles (ngram_range=(1, 2), max_features=100000 but you can try adding more)
    - Time features: publication hour, whether it's morning, day, night, whether it's a weekend
    - Bag of authors (i.e. One-Hot-Encoded author names)

In [12]:
# You code here
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
def read_content(prefix):
    contents = []
    with open(file=PATH_TO_DATA + '/{}_content.txt'.format(prefix), mode='rt', newline='\n') as f:
        for line in f:
            contents.append(line.strip().replace('\xa0', ' ').replace('\u200b', ''))
        f.close()
    return contents

In [14]:
def read_title(prefix):
    with open(PATH_TO_DATA + '/{}_title.txt'.format(prefix), 'rt', newline='\n') as f:
        titles = f.readlines()
        f.close()
    return titles

In [15]:
def read_author(prefix):
    with open(PATH_TO_DATA + '/{}_author.txt'.format(prefix), 'rt') as f:
        authors = f.readlines()
        f.close()
    return authors

In [16]:
def read_date(prefix):
    dates = []
    with open(PATH_TO_DATA + '/{}_published.txt'.format(prefix), 'rt') as f:
        for line in f:
            dates.append(pd.to_datetime(line.strip().replace('$date:','').replace('||', ''), format='%Y-%m-%dT%H:%M:%S'))
        f.close()
    return dates

In [17]:
contents = read_content('train')
titles = read_title('train')
authors = read_author('train')
dates = read_date('train')

In [18]:
len(contents), len(authors), len(titles), len(dates)

(62313, 62313, 62313, 62313)

In [19]:
test_contents = read_content('test')
test_titles = read_title('test')
test_authors = read_author('test')
test_dates = read_date('test')

In [60]:
?TfidfVectorizer

In [68]:
# content_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
# title_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
content_vectorizer = TfidfVectorizer()
title_vectorizer = TfidfVectorizer()
X_train_content_sparse = content_vectorizer.fit_transform(contents)
X_train_title_sparse = title_vectorizer.fit_transform(titles)

In [69]:
all_authors_df = pd.DataFrame({'name':(authors + test_authors)})

In [70]:
len(all_authors_df)

96958

In [71]:
all_id_df = all_authors_df.apply(pd.to_numeric, errors='ignore')

In [72]:
authors_df = pd.DataFrame({'name':authors})

In [130]:
len(authors_df)

62313

In [73]:
id_df = authors_df.apply(pd.to_numeric, errors='ignore')

In [74]:
label_enc = LabelEncoder()
le_dict = {}
le_dict['name'] = label_enc.fit(all_id_df['name'])
id_df['name'] = le_dict['name'].transform(id_df['name'])

In [75]:
all_id_df['name'] = le_dict['name'].transform(all_id_df['name'])

In [76]:
one_hot = OneHotEncoder()
one_hot.fit(all_id_df)

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [77]:
X_train_author_sparse = one_hot.transform(id_df)

In [32]:
X_train_content_sparse.shape

(62313, 22259346)

In [24]:
X_train_title_sparse.shape

(62313, 66021)

In [138]:
X_train_author_sparse.shape

(62313, 45386)

In [33]:
X_train_time_features = pd.DataFrame({'date':dates})

In [34]:
X_train_time_features['hour'] = X_train_time_features['date'].dt.hour
X_train_time_features['month'] = X_train_time_features['date'].dt.month

In [35]:
is_morning = np.logical_and(X_train_time_features['hour'] > 5, X_train_time_features['hour'] <= 11)
X_train_time_features['morning'] = np.where(is_morning, 1, 0)
is_day = np.logical_and(X_train_time_features['hour'] > 11, X_train_time_features['hour'] <= 18)
X_train_time_features['day'] = np.where(is_day, 1, 0)
is_evening = np.logical_and(X_train_time_features['hour'] > 18, X_train_time_features['hour'] <= 24)
X_train_time_features['evening'] = np.where(is_evening, 1, 0)
X_train_time_features['night'] = np.where(X_train_time_features['hour'] <= 5, 1, 0)
X_train_time_features['dow'] = X_train_time_features['date'].dt.dayofweek
is_weekend = np.logical_or(X_train_time_features['dow'] == 5, X_train_time_features['dow'] == 6)
X_train_time_features['weekend'] = np.where(is_weekend, 1, 0)
X_train_time_features['dom'] = X_train_time_features['date'].dt.day
X_train_time_features['business_hour'] = np.where(X_train_time_features['hour'].between(10, 15), 1, 0)
X_train_time_features['winter'] = np.where(X_train_time_features['month'].isin([12, 1, 2]), 1, 0)
X_train_time_features['spring'] = np.where(X_train_time_features['month'].isin([3,4,5]), 1, 0)
X_train_time_features['summer'] = np.where(X_train_time_features['month'].isin([6,7,8]), 1, 0)
X_train_time_features['autumn'] = np.where(X_train_time_features['month'].isin([9,10,11]), 1, 0)

In [36]:
X_train_time_features_sparse = X_train_time_features.drop(['dom', 'dow', 'hour', 'date', 'month'], axis=1)

In [37]:
X_train_time_features_sparse.head()

,morning,day,evening,night,weekend,business_hour,winter,spring,summer,autumn
0,0,0,1,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,0,1,0
2,0,1,0,0,1,1,1,0,0,0
3,1,0,0,0,1,0,0,1,0,0
4,0,1,0,0,1,1,0,0,1,0


**Join all sparse matrices.**

In [39]:
from sklearn.preprocessing import StandardScaler

In [42]:
X_train_content_length = pd.DataFrame({'length': [len(c) for c in contents]})

In [78]:
X_train_content_length.head()

,length
0,-0.338106
1,0.085724
2,-0.605935
3,-0.622848
4,-0.508796


In [44]:
X_train_content_length['length'] = StandardScaler().fit_transform(X_train_content_length[['length']])

In [109]:
X_train_sparse = hstack([X_train_content_sparse, X_train_title_sparse,
                         X_train_author_sparse, 
                         X_train_time_features_sparse]).tocsr()

In [46]:
X_train_sparse.shape

(62313, 22661749)

In [47]:
len(test_contents), len(test_titles), len(test_authors), len(test_dates)

(34645, 34645, 34645, 34645)

In [79]:
X_test_content_sparse = content_vectorizer.transform(test_contents)
X_test_title_sparse = title_vectorizer.transform(test_titles)

In [80]:
test_authors_df = pd.DataFrame({'name': test_authors})
test_id_df = test_authors_df.apply(pd.to_numeric, errors='ignore')
test_id_df['name'] = label_enc.transform(test_id_df['name'])
X_test_author_sparse = one_hot.transform(test_id_df)

In [50]:
X_test_content_sparse.shape, X_test_title_sparse.shape, X_test_author_sparse.shape

((34645, 22259346), (34645, 357006), (34645, 45386))

In [51]:
X_test_time_features = pd.DataFrame({'date':test_dates})
X_test_time_features['hour'] = X_test_time_features['date'].dt.hour
X_test_time_features['month'] = X_test_time_features['date'].dt.month

is_morning = np.logical_and(X_test_time_features['hour'] > 5, X_test_time_features['hour'] <= 11)
X_test_time_features['morning'] = np.where(is_morning, 1, 0)
is_day = np.logical_and(X_test_time_features['hour'] > 11, X_test_time_features['hour'] <= 18)
X_test_time_features['day'] = np.where(is_day, 1, 0)
is_evening = np.logical_and(X_test_time_features['hour'] > 18, X_test_time_features['hour'] <= 24)
X_test_time_features['evening'] = np.where(is_evening, 1, 0)
X_test_time_features['night'] = np.where(X_test_time_features['hour'] <= 5, 1, 0)
X_test_time_features['dow'] = X_test_time_features['date'].dt.dayofweek
is_weekend = np.logical_or(X_test_time_features['dow'] == 5, X_test_time_features['dow'] == 6)
X_test_time_features['weekend'] = np.where(is_weekend, 1, 0)
X_test_time_features['dom'] = X_test_time_features['date'].dt.day
X_test_time_features['business_hour'] = np.where(X_test_time_features['hour'].between(10, 15), 1, 0)
X_test_time_features['winter'] = np.where(X_test_time_features['month'].isin([12, 1, 2]), 1, 0)
X_test_time_features['spring'] = np.where(X_test_time_features['month'].isin([3,4,5]), 1, 0)
X_test_time_features['summer'] = np.where(X_test_time_features['month'].isin([6,7,8]), 1, 0)
X_test_time_features['autumn'] = np.where(X_test_time_features['month'].isin([9,10,11]), 1, 0)
X_test_time_features_sparse = X_test_time_features.drop(['dom', 'dow', 'hour', 'date', 'month'], axis=1)

In [42]:
X_test_time_features_sparse.head()

,morning,day,evening,night,weekend,business_hour,winter,spring,summer,autumn
0,0,1,0,0,0,0,1,0,0,0
1,0,1,0,0,0,1,1,0,0,0
2,0,1,0,0,0,0,1,0,0,0
3,0,1,0,0,0,0,1,0,0,0
4,0,0,1,0,0,0,0,0,0,1


In [52]:
X_test_content_length = pd.DataFrame({'length': [len(c) for c in test_contents]})
X_test_content_length['length'] = StandardScaler().fit_transform(X_test_content_length[['length']])

In [108]:
X_test_sparse = hstack([X_test_content_sparse, X_test_title_sparse,
                        X_test_author_sparse, 
                        X_test_time_features_sparse]).tocsr()

**Read train target and split data for validation.**

In [54]:
X_test_sparse.shape

(34645, 22661749)

In [110]:
train_target = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_log1p_recommends.csv'), 
                           index_col='id')
y_train = train_target['log_recommends'].values

In [111]:
train_part_size = int(0.7 * train_target.shape[0])
X_train_part_sparse = X_train_sparse[:train_part_size, :]
y_train_part = y_train[:train_part_size]
X_valid_sparse =  X_train_sparse[train_part_size:, :]
y_valid = y_train[train_part_size:]

In [57]:
from sklearn.linear_model import LassoCV, RidgeCV, Lasso, Ridge

In [92]:
n_alphas = 20
ridge_alphas = np.logspace(-2, 2, n_alphas)

ridge_cv = RidgeCV(alphas=ridge_alphas, 
                   scoring='mean_absolute_error',
                   cv=3)
ridge_cv.fit(X_train_part_sparse, y_train_part)

/Users/admin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/admin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/admin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/admin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_

/Users/admin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/admin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/admin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/admin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_

/Users/admin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/admin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/admin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/admin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_

/Users/admin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/admin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/admin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/admin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_absolute_error was renamed to neg_mean_absolute_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_

RidgeCV(alphas=array([1.00000e-02, 1.62378e-02, 2.63665e-02, 4.28133e-02, 6.95193e-02,
       1.12884e-01, 1.83298e-01, 2.97635e-01, 4.83293e-01, 7.84760e-01,
       1.27427e+00, 2.06914e+00, 3.35982e+00, 5.45559e+00, 8.85867e+00,
       1.43845e+01, 2.33572e+01, 3.79269e+01, 6.15848e+01, 1.00000e+02]),
    cv=3, fit_intercept=True, gcv_mode=None, normalize=False,
    scoring='mean_absolute_error', store_cv_values=False)

In [103]:
ridge_cv.alpha_

0.01

**Train a simple Ridge model and check MAE on the validation set.**

In [ ]:
# ?RidgeCV

In [97]:
# You code here
ridge_cv.score(X_valid_sparse, y_valid)

0.39133375046067276

**Train the same Ridge with all available data, make predictions for the test set and form a submission file.**

In [91]:
?Ridge

In [112]:
# You code here
ridge = Ridge(random_state=17, alpha=0.01)
ridge.fit(X_train_part_sparse, y_train_part)

Ridge(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=17, solver='auto', tol=0.001)

In [154]:
?ridge.score

In [113]:
ridge_pred = ridge.predict(X_valid_sparse)
valid_mae = mean_absolute_error(y_valid, ridge_pred)
valid_mae, np.expm1(valid_mae)

(1.1871053865511618, 2.277580135511966)

In [104]:
ridge_full = Ridge(random_state=17, alpha=0.01)
ridge_full.fit(X_train_sparse, y_train)

Ridge(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=17, solver='auto', tol=0.001)

In [ ]:
X_train_sparse.shape

In [124]:
X_test_sparse.shape

(34645, 2079421)

In [105]:
ridge_test_pred = ridge_full.predict(X_test_sparse)

In [99]:
# ridge_test_pred = ridge_cv.predict(X_test_sparse)

In [66]:
def write_submission_file(prediction, filename,
                          path_to_sample=os.path.join(PATH_TO_DATA, 
                                                      'sample_submission.csv')):
    submission = pd.read_csv(path_to_sample, index_col='id')
    
    submission['log_recommends'] = prediction
    submission.to_csv(filename)

In [151]:
write_submission_file(ridge_test_pred, os.path.join(PATH_TO_DATA,
                                                    'assignment6_medium_submission.csv'))

**Now's the time for dirty Kaggle hacks. Form a submission file with all zeros. Make a submission. What do you get if you think about it? How is it going to help you with modifying your predictions?**

In [152]:
write_submission_file(np.zeros_like(ridge_test_pred), 
                      os.path.join(PATH_TO_DATA,
                                   'medium_all_zeros_submission.csv'))

**Modify predictions in an appropriate way (based on your all-zero submission) and make a new submission.**

In [182]:
# ridge_test_pred += (len(ridge_test_pred) * 4.33328 - ridge_test_pred.sum()) / len(ridge_test_pred)

In [183]:
ridge_test_pred.sum()

150126.4856

In [173]:
len(ridge_test_pred[ridge_test_pred < 0])

15

In [106]:
ridge_test_pred_modif = ridge_test_pred + (len(ridge_test_pred) * 4.33328 - ridge_test_pred.sum()) / len(ridge_test_pred) # You code here

In [107]:
write_submission_file(ridge_test_pred_modif, 
                      os.path.join(PATH_TO_DATA,
                                   'assignment6_medium_submission_with_hack.csv'))

That's it for the assignment. Much more credits will be given to the winners in this competition, check [course roadmap](https://mlcourse.ai/roadmap). Do not spoil the assignment and the competition - don't share high-performing kernels (with MAE < 1.5).

Some ideas for improvement:

- Engineer good features, this is the key to success. Some simple features will be based on publication time, authors, content length and so on
- You may not ignore HTML and extract some features from there
- You'd better experiment with your validation scheme. You should see a correlation between your local improvements and LB score
- Try TF-IDF, ngrams, Word2Vec and GloVe embeddings
- Try various NLP techniques like stemming and lemmatization
- Tune hyperparameters. In our example, we've left only 50k features and used C=1 as a regularization parameter, this can be changed
- SGD and Vowpal Wabbit will learn much faster
- Play around with blending and/or stacking. An intro is given in [this Kernel](https://www.kaggle.com/kashnitsky/ridge-and-lightgbm-simple-blending) by @yorko 
- In our course, we don't cover neural nets. But it's not obliged to use GRUs/LSTMs/whatever in this competition.

Good luck!

<img src='../../img/kaggle_shakeup.png' width=50%>